In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms,datasets
from torch.utils.data import Dataset,DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from facenet_pytorch import MTCNN

In [2]:
#face detector and mask_classifier
face_classifier =cv2.CascadeClassifier('C:/Users/ashis/Downloads/haar_cascades/haarcascade_frontalface_default.xml')
mtcnn=MTCNN(image_size=240,
            margin=0,
            keep_all=True,
            min_face_size=40)

In [3]:
def preprocess_img(img):
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
    return img

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        model=torchvision.models.mobilenet_v3_large(pretrained=True)
        for param in model.parameters():
            param.requires_grad = False
        self.net=nn.Sequential( model,
                                nn.Linear(1000,1),
                                nn.Sigmoid())
    def forward(self,x):
        return self.net(x)

In [5]:
model=Net()
model=torch.load("mask_classifier.pth")

In [6]:
#testing our model that if it is working well or not.
transform=transforms.Compose([transforms.Resize((224,224)),
                              transforms.ToTensor()])
test_dataset=datasets.ImageFolder("D:/Machine Learning Codes/5.Convolutional Neural Networks/Face Mask Dataset/Test",
                                  transform=transform)
test_dataloader=DataLoader(dataset=test_dataset,
                      batch_size=32,
                      shuffle=True)
#confussion matrix
from sklearn.metrics import confusion_matrix
cm=np.array([[0,0],
             [0,0]])
for i,(x,y) in enumerate(test_dataloader):
    y_pred=torch.round(model(x))
    y=y.reshape((y.shape[0],1)).type(torch.FloatTensor)
    cm+=np.array(confusion_matrix(y,y_pred))
cm

array([[478,   5],
       [  6, 503]])

In [7]:
for i in model.parameters():
    print(i)

Parameter containing:
tensor([[[[-4.8994e-01, -1.9496e-01,  6.9334e-01],
          [-1.1848e+00, -1.3974e-01,  1.4002e+00],
          [-5.9431e-01, -9.0290e-02,  6.3382e-01]],

         [[-1.3701e+00, -3.0764e-01,  1.5945e+00],
          [-2.6857e+00, -3.9656e-01,  3.0067e+00],
          [-1.4104e+00, -1.8335e-01,  1.7502e+00]],

         [[-2.3763e-01, -6.0578e-02,  1.8617e-01],
          [-3.3536e-01, -2.5348e-02,  3.7795e-01],
          [ 8.5835e-04, -2.2011e-02,  1.2646e-01]]],


        [[[-7.2430e-02,  8.8902e-02, -1.3713e-02],
          [ 1.9374e-01,  5.4657e-01,  3.4149e-02],
          [ 5.8198e-02,  1.5683e-02, -3.5735e-02]],

         [[ 1.3941e-01,  4.2239e-01,  3.0752e-01],
          [ 3.9839e-01,  7.3354e-01,  5.3418e-01],
          [ 9.1793e-02,  8.9465e-02,  8.8331e-02]],

         [[-2.3682e-02, -1.1474e-01, -8.4053e-02],
          [ 1.4267e-02,  4.7576e-01, -2.7922e-02],
          [ 4.0951e-02,  2.9826e-02,  4.0062e-02]]],


        [[[ 9.6416e-01,  2.5090e+00,  6.2762

Parameter containing:
tensor([-5.0375e-02, -2.7866e-01,  3.5783e-01, -2.3767e-02,  2.9726e-01,
         5.2534e-02, -5.3232e-01,  6.9528e-10,  3.8491e-02, -8.2901e-01,
        -6.0961e-01, -2.2402e-01, -2.2560e-01,  5.5171e-09,  2.8036e-01,
        -6.2079e-02, -4.7208e-01,  1.7827e-02,  5.6379e-02, -1.7062e-02,
         7.2848e-02, -1.5691e-01,  8.2631e-02, -5.0192e-01,  3.9554e-01,
        -8.8380e-02, -2.0259e-09,  8.1398e-02, -1.3145e-01,  1.2276e-01,
        -5.5950e-01, -7.5223e-01, -1.0281e-02, -5.3982e-01,  2.1001e-01,
         1.7741e-02,  5.9963e-10, -5.1306e-01, -1.4734e-01, -7.8436e-10,
         5.4209e-02, -7.1092e-01, -9.6292e-01,  4.3265e-02,  2.1847e-01,
         2.7183e-01, -9.0520e-01,  3.6238e-02, -1.6246e-02,  3.5136e-02,
         1.9622e-01, -2.9063e-10,  3.2900e-02, -7.7357e-02,  1.4802e-09,
         2.4227e-09,  1.4193e-01,  5.6286e-02,  1.0710e-01,  1.5795e-01,
        -1.2013e+00,  1.1728e-09,  9.7881e-02,  2.8466e-02,  1.4337e-02,
         2.2634e-02, -1.0302e

Parameter containing:
tensor([0.4859, 0.4620, 0.4233, 0.4085, 0.4307, 0.4926, 0.4213, 0.4671, 0.4048,
        0.4746, 0.5765, 0.4303, 0.4388, 0.4141, 0.6877, 0.3918, 0.4102, 0.3797,
        0.4067, 0.3725, 0.4380, 0.4143, 0.4856, 0.3872, 0.4060, 0.4364, 0.4759,
        0.4241, 0.4378, 0.4049, 0.5123, 0.4667, 0.4084, 0.3835, 0.4185, 0.4523,
        0.3837, 0.5988, 0.4726, 0.4308, 0.3995, 0.4657, 0.4966, 0.4620, 0.4414,
        0.4437, 0.5740, 0.4026, 0.4368, 0.3977, 0.4487, 0.4189, 0.4005, 0.6180,
        0.4769, 0.3899, 0.4568, 0.4268, 0.4804, 0.5865, 0.4075, 0.4442, 0.7446,
        0.6787, 0.4112, 0.6293, 0.3868, 0.5314, 0.4110, 0.4309, 0.4471, 0.4156,
        0.6814, 0.5024, 0.4447, 0.5120, 0.6190, 0.4504, 0.4197, 0.4232, 0.3853,
        0.4353, 0.4245, 0.3852, 0.4853, 0.4361, 0.3864, 0.4468, 0.5265, 0.4900,
        0.4356, 0.3783, 0.4372, 0.4724, 0.3753, 0.3988, 0.4847, 0.5461, 0.3890,
        0.3947, 0.4766, 0.5742, 0.6990, 0.3899, 0.4045, 0.4053, 0.4782, 0.4937,
        0.6457, 0.

Parameter containing:
tensor([0.0098])


In [8]:
def predict(img,model):
    transform = transforms.ToTensor()
    inp = transform(img)
    inp = inp.unsqueeze(0)
    output = torch.round(model(inp))
    return output[0][0].item()

In [14]:
video=cv2.VideoCapture(0)
none_obj=None
while True:
    check,frame=video.read()
    gray=preprocess_img(frame)
    #tune the values under detectMultiScale(gray,1.2,5) to get better results
    face_boxes,_=mtcnn.detect(gray)
    if isinstance(face_boxes,type(none_obj)):
        face_boxes=np.array([])
    face_boxes=np.int32(face_boxes)
    for box in face_boxes:
        (x,y,w,h)=box
        cv2.rectangle(frame, (x,y), (w,h), (3, 252, 215), 3)
        pred=predict(frame[y:h, x:w],model)
        if pred==float(0):
            label="Masked"
        else:
            label="Not Masked"
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (3, 252, 215), 2)
    cv2.imshow("Temporary Window",frame)
    key=cv2.waitKey(2)
    if key==27:
        break
video.release()
cv2.destroyAllWindows()